In [34]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [82]:
hungary_gdf = gpd.read_file("data/counties.geojson")

## Get Open Population data

In [83]:
column_names = pd.read_csv(
    "data/stadat-nep0034-22.1.2.1-en.csv", sep=";", encoding="latin-1"
).iloc[0]

In [84]:
raw_population_df = pd.read_csv(
    "data/stadat-nep0034-22.1.2.1-en.csv", sep=";", thousands=" ", encoding="latin-1"
)
raw_population_df.columns = column_names

In [85]:
raw_population_df = raw_population_df.rename(
    columns={"Name of territorial units": "megye"}
)

In [86]:
raw_population_df.index[raw_population_df["megye"] == "Total"]

Index([31, 62, 63, 93], dtype='int64')

In [87]:
raw_population_df = raw_population_df.iloc[64:].reset_index(drop=True)
raw_population_df = raw_population_df.replace(
    {"Gyõr-Moson-Sopron": "Győr-Moson-Sopron", "Csongrád-Csanád": "Csongrád"}
)

In [88]:
hungary_population_gdf = hungary_gdf.merge(raw_population_df, on="megye", how="left")

In [89]:
hungary_population_gdf = hungary_population_gdf[["megye", "geometry", "2014", "2024"]]

In [91]:
hungary_population_gdf["diff"] = (
    hungary_population_gdf["2024"] - hungary_population_gdf["2014"]
)
hungary_population_gdf["percent"] = (
    hungary_population_gdf["diff"] / hungary_population_gdf["2014"]
).round(decimals=2)

In [92]:
hungary_population_gdf

,megye,geometry,2014,2024,diff,percent
0,Budapest,"POLYGON ((18.92617 47.54345, 18.92942 47.54484...",1733837.0,1686222.0,-47615.0,-0.03
1,Baranya,"POLYGON ((17.62507 45.98814, 17.62562 45.98777...",374805.0,353331.0,-21474.0,-0.06
2,Bács-Kiskun,"POLYGON ((18.73307 46.04545, 18.73852 46.04889...",516084.0,491632.0,-24452.0,-0.05
3,Békés,"POLYGON ((20.42439 46.80372, 20.42392 46.80715...",353448.0,310912.0,-42536.0,-0.12
4,Borsod-Abaúj-Zemplén,"POLYGON ((20.05191 48.16772, 20.05288 48.16736...",675767.0,617800.0,-57967.0,-0.09
5,Csongrád,"POLYGON ((19.63818 46.26444, 19.6454 46.27088,...",407148.0,389411.0,-17737.0,-0.04
6,Fejér,"POLYGON ((18.03601 47.33462, 18.03541 47.3369,...",419416.0,419490.0,74.0,0.00
7,Győr-Moson-Sopron,"POLYGON ((16.42153 47.66531, 16.42171 47.66504...",447111.0,473246.0,26135.0,0.06
8,Hajdú-Bihar,"POLYGON ((20.8277 47.65928, 20.83045 47.65733,...",538607.0,520550.0,-18057.0,-0.03
9,Heves,"POLYGON ((19.57113 47.73499, 19.57514 47.73272...",302963.0,285058.0,-17905.0,-0.06


## Merge data and export

In [96]:
population_gdf = bulgaria_gdf.merge(population_df, on="name", how="right")

In [97]:
population_gdf = population_gdf.to_crs(7803)
population_gdf.to_file("data/bulgaria_population.gpkg")